In [2]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [25]:
from git import Repo

In [5]:
from pathlib import Path

In [6]:
PATH_TO_BLOG_REPO = Path('/Users/peterhyland/Documents/GitHub/peter-hyland.github.io/.git')

In [7]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [8]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [9]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [10]:
def update_blog(commit_message='Updates blog'):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [11]:
update_blog()

In [12]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)

    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title

    shutil.copy(cover_image,PATH_TO_CONTENT)

    if not os.path.exists(path_to_new_content): 
        with open(path_to_new_content,'w') as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")

            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br/>\n")
            f.write(f"<h1> {title} <h1>")
            f.write(content.replace("\n", "<br/>\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("blog created")
            return path_to_new_content
    else:
        raise FileExistsError("file already exists")

In [13]:
path_to_new_content = create_new_blog('Test_title', 'rbediewf','logo.png')

blog created


In [14]:
from bs4 import BeautifulSoup as Soup
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [15]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>\n   Document\n  </title>\n</head>\n<body>\n<h1>\n   my blog home\n  </h1>\n<a href="index.html">\n   home\n  </a>\n<a href="content/13.html">\n   13\n  </a>\n<a href="content/14.html">\n   14\n  </a>\n<a href="content/15.html">\n   15\n  </a>\n<a href="content/16.html">\n   16\n  </a>\n</body>\n</html>\n'

In [16]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [17]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    print(links)
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [18]:
write_to_index(path_to_new_content)

[<a href="index.html">
   home
  </a>, <a href="content/13.html">
   13
  </a>, <a href="content/14.html">
   14
  </a>, <a href="content/15.html">
   15
  </a>, <a href="content/16.html">
   16
  </a>]


In [19]:
update_blog()

In [20]:
def create_prompt(title):
    prompt = """
    Biography:
    My name is Jose and I am a python instructor doe coding.

    Blog
    Title: {}
    tags: tech, python, coding, AI, nachine learning
    Summary: I talk about what the future of AI could hold for Python
    Full Text: """.format(title)
    return prompt

In [21]:
title = "The Future of python and AI"
print(create_prompt(title))


    Biography:
    My name is Jose and I am a python instructor doe coding.

    Blog
    Title: The Future of python and AI
    tags: tech, python, coding, AI, nachine learning
    Summary: I talk about what the future of AI could hold for Python
    Full Text: 


In [22]:
response = openai.chat.completions.create(model='gpt-3.5-turbo', 
                                          messages=[{"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}],
                                          max_tokens=1000,
                                          temperature=0.7)


In [23]:
blog_content = response.choices[0].message.content

In [24]:
print(blog_content)

In the world of coding, there lies a trick,
A concept so powerful, it's sure to stick.
It's called recursion, a loop within a loop,
A function that calls itself, a coding coup.

Imagine a function that calls itself again,
Like a mirror reflecting, with no end.
It repeats and repeats, like a never-ending story,
A loop within a loop, recursive glory.

With each iteration, the problem gets smaller,
Solving complex tasks, with a coding chiller.
It's like a Russian doll, nested inside each other,
An elegant solution, like no other.

But beware, for recursion can be a dangerous game,
Infinite loops, with no one to blame.
But when used wisely, it's a programmer's delight,
Solving problems efficiently, with recursive might.

So remember this concept, when coding away,
Recursion in programming, a powerful display.
A loop within a loop, solving problems with ease,
In the world of coding, it's sure to please.
